<a href="https://colab.research.google.com/github/UWI-Capstone-VFalcons/ODModel/blob/main/ARMAPS_real_time_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://cdn-images-1.medium.com/max/1200/1*iDQvKoz7gGHc6YXqvqWWZQ.png" width="85" height="80" >
<br>

#  Custom Real-Time Object Detection

<br>



This notebook is part of the tutorial [Custom real-time object detection in the browser using TensorFlow.js](https://medium.com/@zaninihugo/custom-real-time-object-detection-in-the-browser-using-tensorflow-js-5ca90538eace) and aims to train a custom Object Detection model using the TensorFlow 2 Object Detection API.

Later, this model is going to be converted to TensorFlow.js  to run directly in a web browser. Refer to the tutorial post to see how.

### Installation

### **Clone and install the Tensorflow Object Detection API** 

In order to use the TensorFlow Object Detection API, we need to clone it's GitHub Repo.
<br>

#### **Dependencies**


Most of the dependencies required come preloaded in Google Colab.  No extra installation is needed.
<br>

#### **Protocol Buffers**



The TensorFlow Object Detection API relies on what are called `protocol buffers` (also known as `protobufs`). Protobufs are a language neutral way to describe information. That means you can write a protobuf once and then compile it to be used with other languages, like Python, Java or C [5].

The `protoc` command used below is compiling all the protocol buffers in the `object_detection/protos` folder for Python.

In [ ]:
%cd /content/
!git clone https://github.com/UWI-Capstone-VFalcons/ODModel.git


/content
Cloning into 'ODModel'...
remote: Enumerating objects: 669, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 669 (delta 35), reused 114 (delta 35), pack-reused 554
Receiving objects: 100% (669/669), 494.79 MiB | 33.65 MiB/s, done.
Resolving deltas: 100% (301/301), done.


In [ ]:
%cd /content/ODModel/
!ls

/content/ODModel
armaps-od  Custom_real_time_Object_Detection.ipynb  README.md


In [ ]:
!git pull origin main

From https://github.com/UWI-Capstone-VFalcons/ODModel
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 56219, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 56219 (delta 131), reused 208 (delta 102), pack-reused 55959
Receiving objects: 100% (56219/56219), 572.20 MiB | 32.56 MiB/s, done.
Resolving deltas: 100% (38748/38748), done.


In [ ]:
!ls

armaps-od  Custom_real_time_Object_Detection.ipynb  models  README.md


In [ ]:
%cd /content/ODModel/models/research/
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/ODModel/models/research
Processing /content/ODModel/models/research
     |████████████████████████████████| 9.0MB 18.2MB/s 
     |████████████████████████████████| 358kB 51.4MB/s 
     |████████████████████████████████| 1.1MB 47.4MB/s 
     |████████████████████████████████| 2.3MB 43.3MB/s 
     |████████████████████████████████| 829kB 34.6MB/s 
     |████████████████████████████████| 153kB 52.0MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
     |████████████████████████████████| 17.7MB 251kB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 1.2MB 50.3MB/s 
     |████████████████████████████████| 174kB 58.7MB/s 
     |████████████████████████████████| 645kB 39.7MB/s 
     |████████████████████████████████| 102kB 8.9MB/s 
     |████████████████████████████████| 37.6MB 1.5MB/s 
     |████████████████████████████████| 706kB 43.3MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████

Run the model builder test

In [ ]:
!python /content/ODModel/models/research/object_detection/builders/model_builder_tf2_test.py

2021-04-21 02:09:46.486908: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Running tests under Python 3.7.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2021-04-21 02:09:49.867250: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-21 02:09:49.889630: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-21 02:09:49.976724: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-21 02:09:49.976785: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c2a0c1b62030): /proc/driver/nvidia/version does not exist
2021-04-21 02:09:49.977352: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devic

---

### Getting the data

Creatting the test and train csv files

In [ ]:
%cd /content/ODModel/armaps-od/

/content/ODModel/armaps-od


In [ ]:
!python xml_to_csv.py

Successfully converted xml to csv.
Successfully converted xml to csv.


##### Converting data to TFRecord
The dataset contains two files `train_labels.csv` and `test_labels.csv` which need to be converted into TFRecord format so that it can be fed into Tensorflow’s 2 Object Detection API.

In [ ]:
!python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=records/train.record
!python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=records/test.record

2021-04-21 02:10:25.812532: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Successfully created the TFRecords: /content/ODModel/armaps-od/records/train.record
2021-04-21 02:10:28.801228: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Successfully created the TFRecords: /content/ODModel/armaps-od/records/test.record


---

### Configuring train

We are going to use the pretrained TF2 MobileNet V2 model as the feature extractor in the SSD MobileNet V2 Object Detection model. So the next logical step is to download and untar the pretrained TF2 MobileNet V2 model.

#### Downloading MobileNet weights

In [ ]:
%cd /content/ODModel
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
!tar -xvf mobilenet_v2.tar.gz
!rm mobilenet_v2.tar.gz

/content/ODModel
--2021-04-21 02:10:31--  http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.7.240, 2607:f8b0:4004:802::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.7.240|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8404070 (8.0M) [application/x-tar]
Saving to: ‘mobilenet_v2.tar.gz’

mobilenet_v2.tar.gz 100%[===================>]   8.01M  --.-KB/s    in 0.1s    

2021-04-21 02:10:31 (81.1 MB/s) - ‘mobilenet_v2.tar.gz’ saved [8404070/8404070]

mobilenet_v2/
mobilenet_v2/mobilenet_v2.ckpt-1.index
mobilenet_v2/checkpoint
mobilenet_v2/mobilenet_v2.ckpt-1.data-00001-of-00002
mobilenet_v2/mobilenet_v2.ckpt-1.data-00000-of-00002


In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
!mv ssd_mobilenet_v2_320x320_coco17_tpu-8.config mobilenet_v2.config

--2021-04-21 02:10:31--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4484 (4.4K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.config’

ssd_mobilenet_v2_32 100%[===================>]   4.38K  --.-KB/s    in 0s      

2021-04-21 02:10:31 (48.7 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.config’ saved [4484/4484]



#### Defining training parameters

In [ ]:
num_classes = 9 # number of objects to detect
batch_size = 96 #16
num_steps = 7500 #1500
num_eval_steps = 1000

train_record_path = '/content/ODModel/armaps-od/records/train.record'
test_record_path = '/content/ODModel/armaps-od/records/test.record'
model_dir = '/content/ODModel/armaps-od/training/'
labelmap_path = '/content/ODModel/armaps-od/training/labelmap.pbtxt'

pipeline_config_path = 'mobilenet_v2.config'
fine_tune_checkpoint = '/content/ODModel/mobilenet_v2/mobilenet_v2.ckpt-1'

#### Editing config file

The next cell is modification of the code available at [4].

In [ ]:
import re

with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:

  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
  'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint),
                  config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  f.write(config)

---

### Training the Network

In [ ]:
!python /content/ODModel/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

2021-04-21 02:10:32.504699: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-21 02:10:35.401909: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-21 02:10:35.402922: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-21 02:10:35.414180: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-21 02:10:35.414235: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c2a0c1b62030): /proc/driver/nvidia/version does not exist
2021-04-21 02:10:35.414854: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
W0421 02:10:35.415571 139761606117248 cross_device_ops.py:1321] There are no

---

### Validation

Here we're going yo run the code through a loop that waits for checkpoints to evaluate. Once the evaluation finishes, you're going to see the message:

`INFO:tensorflow:Waiting for new checkpoint at /content/ODModel/armaps-od/training/`

Then you can stop the cell


In [ ]:
!python /content/ODModel/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir}           ##This is passed to run ONLY EVALUATION

2021-04-21 02:13:44.342918: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
W0421 02:13:53.389515 140502694037376 model_lib_v2.py:1063] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0421 02:13:53.389774 140502694037376 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0421 02:13:53.389847 140502694037376 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0421 02:13:53.389915 140502694037376 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0421 02:13:53.390104 140502694037376 model_lib_v2.py:1084] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2021-04-21 02:13:53.429083: I tensorflow/compiler/jit/xla

### Training and validation Metrics 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/ODModel/armaps-od/training/'

Launching TensorBoard...

### Export the Inference Graph

The below code cell adds a line to the tf_utils.py file. This is a temporary fix to a exporting issue occuring when using the OD API with Tensorflow 2. This code will be removed as soon as the OD Team puts out a fix.

All credit goes to the Github users [Jacobsolawetz](https://github.com/Jacobsolawetz) and [ Tanner Gilbert](https://github.com/TannerGilbert), who provided this [temporary fix](https://github.com/tensorflow/models/issues/8841#issuecomment-657647648).

In [ ]:
with open('/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/tf_utils.py') as f:
    tf_utils = f.read()

with open('/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/tf_utils.py', 'w') as f:
  # Set labelmap path
  throw_statement = "raise TypeError('Expected Operation, Variable, or Tensor, got ' + str(x))"
  tf_utils = tf_utils.replace(throw_statement, "if not isinstance(x, str):" + throw_statement)
  f.write(tf_utils)

In [ ]:
output_directory = 'inference_graph'

!python /content/ODModel/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

##### Downloading weights

In [ ]:
!zip -r /content/ODModel/saved_model.zip /content/ODModel/inference_graph/saved_model/

In [ ]:
from google.colab import files
files.download("/content/ODModel/saved_model.zip")

---

###  Testing the trained model

Based on [Object Detection API Demo](https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb) and [Inference from saved model tf2 colab](https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/inference_from_saved_model_tf2_colab.ipynb).

In [ ]:
%cd /content/ODModel

In [ ]:
!wget https://raw.githubusercontent.com/hugozanini/object-detection/master/inferenceutils.py
from inferenceutils import *

##### Loading the model

In [ ]:
output_directory = 'inference_graph/'

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'/content/ODModel/{output_directory}/saved_model')

##### Selecting the images to test

In [ ]:
import pandas as pd
test = pd.read_csv('/content/ODModel/armaps-od/images/test_labels.csv')
images = list(test['filename'][0:20])

##### Doing inference

In [ ]:
#images = ['canguru-1.jpeg', 'canguru-2.jpeg', 'canguru-3.jpeg']

for image_name in images:
  
  image_np = load_image_into_numpy_array('/content/ODModel/armaps-od/images/test/' + image_name)
  output_dict = run_inference_for_single_image(model, image_np)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  display(Image.fromarray(image_np))

---

### References

[0] Tanner, G. (2020, July 27). Tensorflow Object Detection with Tensorflow 2: Creating a custom model. Retrieved December 28, 2020, from https://gilberttanner.com/blog/tensorflow-object-detection-with-tensorflow-2-creating-a-custom-model

---

[1] Rafiq, H. (2020, December 13). Image Object Detection - TensorFlow 2 Object Detection API. Retrieved December 28, 2020, from https://medium.com/swlh/image-object-detection-tensorflow-2-object-detection-api-af7244d4c34e

---

[2] TensorFlow 2 Detection Model Zoo. (2020, September 09). Tensorflow/models. Retrieved December 28, 2020, from https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

---

[3] Tensorflow 2 Models Configs. (2020, August 11). Tensorflow/models. Retrieved December 28, 2020, from https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2

---
[4] Solawetz, J. (2020, December 09). How to Train YOLOv4 on a Custom Dataset. Retrieved December 28, 2020, from https://blog.roboflow.com/training-yolov4-on-a-custom-dataset/

---

[5] Google Colaboratory. (n.d.). Retrieved January 13, 2021, from https://colab.research.google.com/github/cloud-annotations/google-colab-training/blob/master/object_detection.ipynb#:~:text=The%20TensorFlow%20Object%20Detection%20API%20relies%20on%20what%20are%20called,like%20Python%2C%20Java%20or%20C.
